In [79]:
from hanoi_states import StatesHanoi
from hanoi_states import ProblemHanoi
from tree_hanoi import NodeHanoi


----

## Búsqueda voraz (greedy) primero el mejor


### Definimos una Priority Queue

In [80]:
import heapq

class PriorityQueue:
    def __init__(self, priority_function):
        self.priority_function = priority_function
        self.heap = []
        self.counter = 0  # A counter to handle items with the same priority

    def push(self, item):
        # Create a tuple of (priority, counter, item)
        # The counter ensures the items are ordered correctly if they have the same priority
        priority = self.priority_function(item)
        heapq.heappush(self.heap, (priority, self.counter, item))
        self.counter += 1

    def pop(self):
        # Pop the item with the lowest priority (highest priority in terms of the queue)
        if self.heap:
            return heapq.heappop(self.heap)[2]
        raise IndexError("pop from an empty priority queue")

    def is_empty(self):
        return len(self.heap) == 0
    
    def __len__(self):
        return len(self.heap)

### Definimos una función heurística

In [81]:
def heuristic_func(new_node):
    last_rod_len = len(new_node.state.rods[len(new_node.state.rods) - 1])
    return last_rod_len * -1

In [82]:
def greedy_search(initial_node):
    frontier = PriorityQueue(heuristic_func)
    frontier.push(initial_node)
    
    explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente
    # Mientras que la cola no este vacia
    while len(frontier) != 0:
        node = frontier.pop()  # Extraemos el primer nodo de la cola (Más prioritario)
        
        # Agregamos nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
            return node, explored, frontier
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node.expand(problem):
            # Solo si no fue explorado
            if next_node.state not in explored:
                heuristic_func(next_node)
                frontier.push(next_node) # La Priority Queue se encarga de ordenar la cola a medida que se inserta utilizando la funcion heuristica dada

In [83]:
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

last_node, explored, frontier = greedy_search(NodeHanoi(problem.initial))

In [84]:
print(f'Longitud del camino de la solución: {last_node.state.accumulated_cost}')

Longitud del camino de la solución: 35.0


In [85]:
print(len(explored), "nodos se expandieron y", len(frontier), "nodos quedaron en la frontera")

168 nodos se expandieron y 39 nodos quedaron en la frontera


In [86]:
node = last_node
while node.parent is not None:
    print(node.state)
    node = node.parent

HanoiState:  |  | 5 4 3 2 1
HanoiState: 1 |  | 5 4 3 2
HanoiState: 1 | 2 | 5 4 3
HanoiState:  | 2 1 | 5 4 3
HanoiState: 3 | 2 1 | 5 4
HanoiState: 3 | 2 | 5 4 1
HanoiState: 3 2 |  | 5 4 1
HanoiState: 3 2 1 |  | 5 4
HanoiState: 3 2 1 | 4 | 5
HanoiState: 3 2 | 4 | 5 1
HanoiState: 3 | 4 2 | 5 1
HanoiState: 3 1 | 4 2 | 5
HanoiState: 3 1 | 4 | 5 2
HanoiState: 3 | 4 | 5 2 1
HanoiState:  | 4 3 | 5 2 1
HanoiState: 1 | 4 3 | 5 2
HanoiState: 1 | 4 3 2 | 5
HanoiState:  | 4 3 2 1 | 5
HanoiState: 5 | 4 3 2 1 | 
HanoiState: 5 | 4 3 2 | 1
HanoiState: 5 2 | 4 3 | 1
HanoiState: 5 2 1 | 4 3 | 
HanoiState: 5 2 1 | 4 | 3
HanoiState: 5 2 | 4 | 3 1
HanoiState: 5 | 4 2 | 3 1
HanoiState: 5 1 | 4 2 | 3
HanoiState: 5 1 | 4 | 3 2
HanoiState: 5 | 4 | 3 2 1
HanoiState: 5 4 |  | 3 2 1
HanoiState: 5 4 1 |  | 3 2
HanoiState: 5 4 1 | 2 | 3
HanoiState: 5 4 | 2 1 | 3
HanoiState: 5 4 3 | 2 1 | 
HanoiState: 5 4 3 | 2 | 1
HanoiState: 5 4 3 2 |  | 1


In [87]:
%%timeit 
# Inicializaos el problema

initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

greedy_search(NodeHanoi(problem.initial))

12.3 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Vemos que la solución demoro en promedio 12.5ms. 

In [88]:
import tracemalloc

# Para medir memoria consumida (usamos el pico de memoria)
tracemalloc.start()

# Inicializaos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

greedy_search(NodeHanoi(problem.initial))
            
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Maxima memoria ocupada: {round(memory_peak, 2)} [MB]", )

Maxima memoria ocupada: 0.25 [MB]


Maxima memoria ocupada: 0.25 [MB]